# Pure Python

In [1]:
def pure_python_simple(filename, maketuple=False):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    if maketuple:
        full_dict = tuple(full_dict.values())
    with open(filename, 'rb') as f:
        print(sum(full_dict[i] for i in f.read()))

In [2]:
%%timeit -n 1 -r 1
pure_python_simple("random-20221005.txt")

9140634224
7.31 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [3]:
%%timeit -n 1 -r 1
pure_python_simple("random-20221005.txt", maketuple=True)

9140634224
7.17 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Numpy

In [4]:
import numpy as np
def numpy_indexing(filename, do_loops=False):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.uint32)
    with open(filename, 'rb') as f:
        if do_loops:
            total = 0
            temp = 1
            loops = 0
            while temp:
                temp = np.sum(full_dict[np.frombuffer(f.read1(100000), dtype=np.uint8)])
                total += temp
                loops += 1
            print(total)
            print(loops)
        else:
            print(np.sum(full_dict[np.frombuffer(f.read(), dtype=np.uint8)]))

In [5]:
#Error
np.sum(np.array([800]*10000000))

-589934592

In [6]:
%%timeit -n 1 -r 1
numpy_indexing("random-20221005.txt")

550699632
527 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
%%timeit
numpy_indexing("random-20221005.txt", do_loops=True)

9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
359 ms ± 23.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Pure Numba

In [8]:
from numba import jit
@jit(nogil=True, fastmath=True)
def gematria_sum(bytearr):
    total = 0
    for char in bytearr:
        if char < 65:
            continue
        if char > 96:
            char -= 32
        if char < 74: #A 4- J
            total += char-64
            continue
        if char < 84: #K - S
            total += (char-73)*10
            continue
        if char < 91: #T - Z
            total += (char-82)*100
    return total
gematria_sum(b'qwertyuiopasdfghjklzxcvbnm') #to compile

4095

In [9]:
def numba_basic(filename, do_loops=False):
    with open(filename, 'rb') as f:
        if do_loops:
            total = 0
            temp = 1
            loops = 0
            while temp:
                temp = gematria_sum(f.read1(100000))
                total += temp
                loops += 1
            print(total)
            print(loops)
        else:
            print(gematria_sum(f.read()))

In [10]:
%%timeit -n 1 -r 1
numba_basic("random-20221005.txt")

9140634224
841 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
%%timeit -n 1
numba_basic("random-20221005.txt", do_loops=True)

9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
830 ms ± 177 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Numba + Numpy

In [12]:
#from numba import jit
import numba
numba.jit(nogil=True, fastmath=True)
def np_index_sum(full_dict, bytearr):
    return np.sum(full_dict[np.frombuffer(bytearr, dtype=np.uint8)])
np_index_sum(np.array([2,2], dtype=np.uint32), b'\x00') #to compile

2

In [13]:
import numpy as np
def numba_numpy_indexing(filename, do_loops=False):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.uint32)
    with open(filename, 'rb') as f:
        if do_loops:
            total = 0
            temp = 1
            loops = 0
            while temp:
                temp = np_index_sum(full_dict, f.read1(100000))
                total += temp
                loops += 1
            print(total)
            print(loops)
        else:
            print(np_index_sum(full_dict, f.read()))

In [14]:
%%timeit -n 1 -r 1
numba_numpy_indexing("random-20221005.txt")

550699632
840 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit
numba_numpy_indexing("random-20221005.txt", do_loops=True)

9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
361 ms ± 54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
def only_read(filename):
    with open(filename, 'rb') as f:
        while len(f.read1(1000000)):
            pass

In [17]:
%%timeit
only_read("random-20221005.txt")

23.2 ms ± 342 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
import threading

def threaded_numpy(filename):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.uint32)
    
    lock = threading.Lock()
    results = []
    def do_job(full_dict, bytearr, results, lock):
        num = np.sum(full_dict[np.frombuffer(bytearr, dtype=np.uint8)])
        with lock:
            results.append(num)
    threads = []
    
    with open(filename, 'rb') as f:
        total = 0
        loops = 0
        temp = f.read1(1000000)
        while len(temp):
            threads.append(threading.Thread(target=do_job, 
                                            args=(full_dict, temp, results, lock)))
            threads[-1].start()
            loops += 1
            temp = f.read1(1000000)
            if len(threads)>10:
                threads[0].join()
                del threads[0]
        for thread in threads:
            thread.join()
        print(sum(results))
        print(loops)

In [19]:
%%timeit -n 1
threaded_numpy("random-20221005.txt")

9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
191 ms ± 5.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
from concurrent.futures import ThreadPoolExecutor

def threaded_new_numpy(filename):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.uint32)

    with open(filename, 'rb') as f:
        loops = 0
        temp = f.read1(1000000)
        threads = []
        with ThreadPoolExecutor(max_workers=10) as executor:
            while len(temp):
                threads.append(executor.submit(np_index_sum, full_dict, temp))
                temp = f.read1(1000000)
                loops += 1
        print(sum(thread.result() for thread in threads))
        print(loops)

In [21]:
%%timeit -n 1
threaded_new_numpy("random-20221005.txt")

9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
194 ms ± 6.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# FINAL

In [22]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def final_fast_gematria(filename, chunksize=1000000, max_threads=20):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.uint32)
    task = lambda full_dict, bytearr: np.sum(full_dict[np.frombuffer(bytearr, dtype=np.uint8)])
    threads = []
    with open(filename, 'rb') as f:
        with ThreadPoolExecutor(max_workers=max_threads) as executor:
            temp = f.read1(chunksize)
            while len(temp):
                threads.append(executor.submit(task, full_dict, temp))
                temp = f.read1(chunksize)
    return sum(thread.result() for thread in threads)

In [23]:
%%timeit -n 1
print(final_fast_gematria("random-20221005.txt"))

9140634224
9140634224
9140634224
9140634224
9140634224
9140634224
9140634224
193 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
